## Segments

Action Sets? `./SET/*.BIN` and `SET.MUK`

## action sets

describe which set of sprites refer to a certain action, game files has a couple of action set BIN files, the rest are merged into one big SET.MUK file

Action files for regular sprites are laid out in the following format:
```
ACTION_FRAME_COUNT   POSITION_FORMAT
X_COORD Y_COORD CEL_NUMBER FLIP TIME_IN_TICKS {PAGE_NUMBER}
X_COORD Y_COORD CEL_NUMBER FLIP TIME_IN_TICKS {PAGE_NUMBER}
```
—and so on, where each new line is a new frame.
       Notes
    • POSITION_FORMAT can be R or A; for sprites, however, it is seemingly always set to R.
    • FLIP must be either 1 or -1.
    
## for large composite sprites:

Action files for boss sprites are laid out in the following format:
```
ACTION_FRAME_COUNT   POSITION_FORMAT   SPRITE_COUNT
PART_NO X Y CEL_NO FLIP Z_HGT TICKS {DAMAGED} {PAGE_NUMBER}
PART_NO X Y CEL_NO FLIP Z_HGT TICKS {DAMAGED} {PAGE_NUMBER}
```
—and so on, where each new line is a new frame.
       Notes
    • For boss sprites, POSITION_FORMAT is B.
    • FLIP must be either 1 or -1.
    
A frame is now defined by multiple parts, where frames are delimited by a new zero part.
For parts other than a zero part (that is, the start of a new frame), the x and y values are the deltas (changes) relative to the zero part that define where this part is placed.
A Damaged version is specified by a 1, which defaults to zero if not damaged.
Huh? What does that (above) mean?
TIME_IN_TICKS is irrelevant for cels beyond the first one in the frame; however, you must enter it as a place holder if specifying the part is a Damaged one or the cel is on a different page. The same applies to the Damaged flag if specifying a different page number.
ACTION_FRAME_COUNT is the number of actual frames, not the total number of lines in the file.
If there is a Damaged cel version, it must be listed before the normal cel or MAKESET will issue an error.
### Coding Events
Events are actions such as sprite shooting or making contact with a sword. They are coded in as a line in the Action files by setting the CEL_NUMBER value to -1 and the ticks to zero.

In [2]:
import os
from collections import defaultdict
import logging
from IPython.display import HTML, display
import math
from itertools import islice, product
from enum import Enum

In [7]:
# tilesets; DICT[str, Tileset]
%run ./CEL\ INSPECTOR.ipynb

In [4]:
dir()

['CEL_FOLDER',
 'Enum',
 'HTML',
 'Image',
 'In',
 'OUTPUT',
 'Out',
 'Sprite',
 '_',
 '_1',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'bounds',
 'chunk',
 'convert_files',
 'convert_pixel',
 'defaultdict',
 'display',
 'display_hex_view',
 'exit',
 'format_word',
 'get_ipython',
 'hex_view',
 'islice',
 'itertools',
 'log_progress',
 'logging',
 'math',
 'namedtuple',
 'os',
 'pairwise',
 'product',
 'quit',
 'set_pixel',
 'sprite_header',
 'tripwise',
 'twos']

In [3]:
def chunk(it, size):
    it = iter(it)
    return list(iter(lambda: tuple(islice(it, size)), ()))


def format_word(v, w):
    text = '{:02X} {:02X}'.format(v, w)
    return f'{text}'

    
def hex_view(data):
    text = []
    text.append(
        14 * '&nbsp;' +
        '&nbsp;'.join(['{:02X}'.format(y) for y in range(8)]) +
        '&nbsp;&nbsp;' +
        '&nbsp;'.join(['{:02X}'.format(y) for y in range(8, 16)])
    
    )
    text.append('')
    for i, x in enumerate(chunk(data, 16)):
        text.append('{:08X}:&nbsp;&nbsp;&nbsp;&nbsp;'.format(i * 16) +
              '&nbsp;'.join([format_word(y, z) for y, z in zip(x[:8:2], x[1:8:2])]) +
              '&nbsp;&nbsp;' +
              '&nbsp;'.join([format_word(y, z) for y, z in zip(x[8:16:2], x[9:16:2])])
             )
    return '<span style="font-family: Courier New, Courier, Lucida Sans Typewriter, Lucida Typewriter, monospace;">{}</span>'.format("<br>".join(text))


def display_hex_view(data):
    display(HTML(hex_view(data)))


#display_hex_view(bytes(range(132)))

In [11]:
# Look at the top of the files
FOLDER = '/home/alex/.wine/drive_c/GOG Games/WH40K Final Liberation/SET'
OUTPUT = './sprites/'

for filepath in sorted(os.listdir(FOLDER)):
    filesize = os.path.getsize(FOLDER + '/' + filepath)
    filename = filepath.split('.')[0]
    print(filename, filesize)

SET2102 4654
SET2108 5806


In [60]:
from itertools import zip_longest


def read_file(file):
    sprite_number = filepath.split('.')[0].replace('SET', '')
    base_sprite = Sprite(f'PAGE{sprite_number}.CEL')
    with open(f'{FOLDER}/{file}', 'rb') as file:
        header = file.read(0x28)
        print("unknown1", int.from_bytes(file.read(2), 'little', signed=True))
        headers = int.from_bytes(file.read(2), 'little', signed=True)
        print("sets:", headers)
        print("unknown3", int.from_bytes(file.read(2), 'little', signed=True))
        h = []
        for i in range(headers):
            # at least part of the second value is an index?
            h.append([
                int.from_bytes(file.read(4), 'little', signed=True),
                int.from_bytes(file.read(4), 'little', signed=True)
            ])
        prev = 0
        #print(h)
        header_size = 0x28 + 6 + 8 * headers
        #print(h[-1][0], os.fstat(file.fileno()).st_size - header_size)
        #print()
        for set_index, (start, end) in enumerate(zip_longest(h[0:], h[1:])):
            if end is None:
                end = [os.fstat(file.fileno()).st_size]
            # display_hex_view(data)
            #print("set size", end[0] - start[0])
            #file.seek(start[0] + header_size - 2)
            set_frames = int.from_bytes(file.read(2), 'little')
            #print(set_index, "set frames", set_frames)
            frames = []
            for i in range(set_frames):
                if i == 0:
                    assert int.from_bytes(file.read(2), 'little') == 256, "Invalid start of set"
                frame_data = [
                        int.from_bytes(x, 'little', signed=True) 
                        for x in chunk(file.read(16), 2)
                    ]
                #print(frame_data)
                if frame_data[2] != -1 and frame_data[0] == int(sprite_number):
                    try:
                        frames.append(
                            base_sprite.images[frame_data[1]]
                            if frame_data[3] == 1 else
                            base_sprite.images[frame_data[1]].transpose(Image.FLIP_LEFT_RIGHT)
                        )
                    except KeyError as e:
                        print("Invalid CEL index {} of {}".format(frame_data[1], len(base_sprite.images)))
                elif frame_data[0] != int(sprite_number):
                    raise RuntimeError("Not yet got other CEL files getting loaded!")
            
            if not os.path.exists(f'gifs/{sprite_number}'):
                os.makedirs(f'gifs/{sprite_number}')
            frames[0].save(f'./gifs/{sprite_number}/{sprite_number}-{set_index}.gif', format='GIF',
               append_images=frames[1:], save_all=True, duration=100, loop=0)
        pos, size = file.tell(), os.fstat(file.fileno()).st_size
        assert pos == size , f"Did not read to EOF {pos} of {size}"

In [61]:
  #     frames[0].save('test.gif', format='GIF',
    #                append_images=frames[1:], save_all=True, duration=10, loop=0)

In [62]:
for filepath in sorted(os.listdir(FOLDER)):
    print(filepath)
    read_file(filepath)

SET2102.BIN
unknown1 216
sets: 96
unknown3 1
Invalid CEL index 29 of 29
Invalid CEL index 29 of 29
SET2108.BIN
unknown1 294
sets: 88
unknown3 1
Invalid CEL index 32 of 32
Invalid CEL index 32 of 32


In [8]:
# boss set has 9 variables
# regular set has 6

In [9]:
# PAGE_NUMBER CEL_NO TIME_IN_TICKS FLIP Y X PART? ?DAMAGED
# No room for Z_HGT???

In [10]:
# ACTION_FRAME_COUNT   POSITION_FORMAT   SPRITE_COUNT
# PART_NO X Y CEL_NO FLIP Z_HGT TICKS {DAMAGED} {PAGE_NUMBER}
# PART_NO X Y CEL_NO FLIP Z_HGT TICKS {DAMAGED} {PAGE_NUMBER}